In [34]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.optim as optim
import math
from tensorboardX import SummaryWriter

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

REPORT_EVERY_ITER = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device', device)

('using device', device(type='cpu'))


In [35]:
dataset = ControlsDataset()
print("Stack size", dataset.stack_size)
# dataloader = DataLoader(dataset, batch_size = 256, shuffle = True, num_workers = 0)
net = ConvNet().to(device)

print("Data size", dataset.data_frame.size)
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

('Stack size', 1)
('units after conv', 512)
('conv parameters: ', 168224)
('fc parameters: ', 164353)
('Data size', 11018)
('number of parameters: ', 332577)


In [36]:
torch.cuda.empty_cache()

In [37]:
validation_split = .2
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

# spliting the dataset
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


# Training data loader # NOTE had to remove shuffle
dataloader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=train_sampler)

# Validation data loader # NOTE had to remove shuffle
validLoader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=valid_sampler)

print("Total training stacks", len(dataloader))
print("Total validation stacks",len(validLoader))

('Total training stacks', 35)
('Total validation stacks', 9)


In [38]:
epochs = 25
optimizer = optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss() # Changed from Mean-Squared to Cross-Entropy
writer = SummaryWriter()
iter_no = 0

torch.cuda.empty_cache()
for epoch in range(epochs):
    for i_batch, sampled_batch in enumerate(dataloader):
        iter_no += 1
        images = sampled_batch['image'].to(device).float()
        controls = sampled_batch['control'].to(device).float()
    
        optimizer.zero_grad()
        prediction = net(images)
#         print((prediction[0], controls))
        loss = criterion(prediction, controls)
        loss.backward()
        optimizer.step()

        if iter_no % REPORT_EVERY_ITER == 0:
            writer.add_scalar("Loss", loss.item(), iter_no)
            torch.save(net.state_dict(), "snapshots/saved_model_{}".format(loss.item()))
        
        out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB\n"
        print(out.format(str(epoch),
                        str(iter_no),
                        round(loss.item(),5),
                         'na', 'na'
#                         round(torch.cuda.memory_allocated(0)/1024**3,3),
#                         round(torch.cuda.memory_allocated(0)/1024**3,3)
                        ))
        torch.cuda.empty_cache()

RuntimeError: 1D target tensor expected, multi-target not supported

In [26]:
# Validation Test
iter_no = 0
for i_batch, sampled_batch in enumerate(validLoader):
    iter_no += 1
    images = sampled_batch['image'].to(device).float()
    controls = sampled_batch['control'].to(device).float()

    optimizer.zero_grad()
    prediction = net(images)
    loss = criterion(prediction, controls)
    out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB\n"
    print(out.format(str(epoch),
                    str(iter_no),
                    round(loss.item(),5),
                     'na', 'na'
#                         round(torch.cuda.memory_allocated(0)/1024**3,3),
#                         round(torch.cuda.memory_allocated(0)/1024**3,3)
                    ))
    torch.cuda.empty_cache()

6,1	Loss:0.11308	Allocated:naGB	Cached:naGB

6,2	Loss:0.1104	Allocated:naGB	Cached:naGB

6,3	Loss:0.10524	Allocated:naGB	Cached:naGB

6,4	Loss:0.12263	Allocated:naGB	Cached:naGB

6,5	Loss:0.15256	Allocated:naGB	Cached:naGB

6,6	Loss:0.08564	Allocated:naGB	Cached:naGB

6,7	Loss:0.11901	Allocated:naGB	Cached:naGB

6,8	Loss:0.13282	Allocated:naGB	Cached:naGB

6,9	Loss:0.15726	Allocated:naGB	Cached:naGB



In [ ]:
torch.cuda.empty_cache()

In [4]:
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024))

GeForce GTX 1050
Memory Usage:
Allocated: 0.0 GB
Cached:    2048
